Импортируем библиотеки, грузим датасеты

In [6]:
# Импортируем библиотеки

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sb

from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestClassifier

# Грузим датасеты

df_train = pd.read_csv('data/train_processed.csv', low_memory=False)
df_valid = pd.read_csv('data/valid_processed.csv', low_memory=False)

Baseline модель

In [7]:
# Разделяем на таргет и параметры
X_train = df_train.drop('target', axis=1)
y_train = df_train['target']

groups = df_train['client_id']

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in gss.split(X_train, y_train, groups=groups):
    X_train, X_test = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train, y_test = y_train.iloc[train_idx], y_train.iloc[test_idx]

assert X_train['client_id'].isin(X_test['client_id']).sum() == 0

X_valid = df_valid.drop('target', axis=1)
y_valid = df_valid['target']

# Обучаем модель на train
model = RandomForestClassifier(
    n_estimators=35,
    max_depth=None,
    min_samples_split=18,
    min_samples_leaf=7,
    max_features='sqrt',
    criterion='gini',
    class_weight='balanced',
    random_state=42
)
model.fit(X_train.drop('client_id', axis=1), y_train)

In [8]:
y_pred_proba_test = model.predict_proba(X_test.drop('client_id', axis=1))[:, 1]
roc_auc_test = roc_auc_score(y_test, y_pred_proba_test)
print(f"ROC-AUC on test data: {roc_auc_test}")
fpr_test, tpr_test, _ = roc_curve(y_test, y_pred_proba_test)

precision_test, recall_test, thresholds_test = precision_recall_curve(y_test, y_pred_proba_test)
average_precision_test = average_precision_score(y_test, y_pred_proba_test)
print(f'Precission-Recall on test data: {average_precision_test}')

y_pred_proba = model.predict_proba(X_valid.drop('client_id', axis=1))[:, 1]
roc_auc = roc_auc_score(y_valid, y_pred_proba)
print(f"ROC-AUC on validation data: {roc_auc}")
fpr_valid, tpr_valid, _ = roc_curve(y_valid, y_pred_proba)

precision, recall, thresholds = precision_recall_curve(y_valid, y_pred_proba)
average_precision = average_precision_score(y_valid, y_pred_proba)
print(f'Precission-Recall on validation data: {average_precision}')

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.